# 🔍 RAG with ChromaDB

**A Complete RAG System for Training Sessions**

This notebook demonstrates:
- **ChromaDB** for vector storage and retrieval
- **Google Gemini** for response generation
- **Simple setup** with minimal code
- **Interactive examples** for learning

## 🎯 What You'll Learn

1. **Document Storage**: Creating and populating a knowledge base with ChromaDB
2. **Semantic Search**: Finding relevant documents using vector similarity
3. **Context Retrieval**: Extracting relevant context for response generation
4. **RAG Pipeline**: Building a complete Retrieval-Augmented Generation system
5. **Interactive Queries**: Practicing with real RAG queries

---

## 📊 Project Overview

**Goal:** Build a functional RAG system that can answer questions based on a knowledge base.

**Key Components:**
- 📚 **Knowledge Base**: Storing documents with embeddings
- 🔍 **Vector Search**: Finding semantically similar documents
- 🤖 **LLM Generation**: Creating answers from retrieved context
- 🔄 **RAG Pipeline**: Complete retrieval and generation workflow

---

## What is RAG?

**Retrieval-Augmented Generation (RAG)** combines:
1. **Document Retrieval**: Finding relevant information from a knowledge base
2. **Response Generation**: Using retrieved context to generate accurate answers

**Why ChromaDB?**
- Easy to use vector database
- Built-in embedding generation
- Efficient similarity search
- Perfect for prototyping

---

## ⚙️ Section 1: Setup and Installation


##  Setup and Installation

In [2]:
# Install required packages
!pip -q install chromadb

In [42]:
# Import required libraries
import chromadb
import google.generativeai as genai
from google.genai import types
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
import json
from typing import List, Dict
import os

## API Configuration

In [55]:
# Configure Gemini API
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY_1')
    genai.configure(api_key=GEMINI_API_KEY)
    print("Gemini API configured from Colab secrets!")
except:
    print("No API key found in secrets.")

gemini_model = "gemini-2.5-flash"

No API key found in secrets.


## ChromaDB Setup

In [5]:
# Initialize ChromaDB client
client = chromadb.Client()

# Create or get a collection
collection_name = "rag_knowledge_base"
try:
    # Try to get existing collection
    collection = client.get_collection(name=collection_name)
    print(f"Using existing collection: {collection_name}")
except:
    # Create new collection with default embedding function
    collection = client.create_collection(
        name=collection_name,
        metadata={"description": "RAG Knowledge Base for Training"}
    )
    print(f"Created new collection: {collection_name}")

print(f"ChromaDB collection ready!")
print(f"Collection: {collection.name}")
print(f"Count: {collection.count()} documents")

Created new collection: rag_knowledge_base
ChromaDB collection ready!
Collection: rag_knowledge_base
Count: 0 documents


## Raw Data

In [11]:
data = """Artificial Intelligence (AI) is a branch of computer science focused on creating intelligent machines that can perform tasks typically requiring human intelligence. AI systems can learn, reason, and make decisions.


Machine Learning is a subset of AI that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms to find patterns in data and make predictions.


Deep Learning uses artificial neural networks with multiple layers to model complex patterns in data. It's particularly effective for image recognition, natural language processing, and speech recognition.


RAG (Retrieval-Augmented Generation) combines information retrieval with text generation. It searches a knowledge base for relevant documents, then uses this context to generate accurate, informed responses.


Vector embeddings are numerical representations that capture the semantic meaning of text. Similar concepts have similar embeddings, enabling semantic search and similarity comparisons.


ChromaDB is an open-source vector database designed for AI applications. It provides easy-to-use APIs for storing, querying, and managing embeddings with built-in similarity search.


Python is a popular programming language for AI and data science due to its simplicity, extensive libraries like NumPy and TensorFlow, and strong community support.


Natural Language Processing (NLP) enables computers to understand and generate human language. It includes tasks like text analysis, translation, sentiment analysis, and question answering.


Google Colab is a free cloud-based Jupyter notebook environment with GPU/TPU access. It's perfect for machine learning experiments and comes with pre-installed AI libraries.


Large Language Models (LLMs) like GPT and Gemini are AI models trained on vast amounts of text. They can understand context, generate human-like text, and perform various language tasks."""

## Data Chunking

In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,    # max tokens/characters per chunk
    chunk_overlap=0,  # overlap between chunks
    separators=["\n\n"]  # prioritize splitting by paragraph breaks
)

chunks = text_splitter.split_text(data)

len(chunks) # number of chunks

10

## Knowledge Base Creation

In [102]:
# Although an LLM can add metadata, we'll do it manually to save Gemini AI resources.

metadata_list = [
    ("AI Fundamentals", "artificial_intelligence"),
    ("AI Fundamentals", "machine_learning"),
    ("AI Fundamentals", "deep_learning"),
    ("NLP Techniques", "rag"),
    ("NLP Techniques", "embeddings"),
    ("Tools", "chromadb"),
    ("Programming", "python"),
    ("NLP Techniques", "nlp"),
    ("Tools", "colab"),
    ("AI Fundamentals", "llm")
]

documents = []
for i, (chunk, metadata) in enumerate(zip(chunks, metadata_list)):
    category, topic = metadata

    # Create the document dictionary
    documents.append({
        "id": f"doc_{i + 1}",
        "text": chunk.strip(),
        "category": category,
        "topic": topic
    })

In [107]:
# print sample document
documents[0]

{'id': 'doc_1',
 'text': 'Artificial Intelligence (AI) is a branch of computer science focused on creating intelligent machines that can perform tasks typically requiring human intelligence. AI systems can learn, reason, and make decisions.',
 'category': 'AI Fundamentals',
 'topic': 'artificial_intelligence'}

## Knowledge Base Creation

In [93]:
# Add documents to ChromaDB collection

# Only add if collection is empty
if collection.count() == 0:
    # Prepare data for ChromaDB
    texts = [doc["text"] for doc in documents]
    ids = [doc["id"] for doc in documents]
    metadatas = [{"category": doc["category"], "topic": doc["topic"]} for doc in documents]

    # Add to collection (ChromaDB will automatically generate embeddings)
    collection.add(
        documents=texts,
        ids=ids,
        metadatas=metadatas
    )

    print(f"Added {len(documents)} documents to ChromaDB")
else:
    print(f"Collection already has {collection.count()} documents")

print(f"Knowledge base ready with {collection.count()} documents!")

Collection already has 11 documents
Knowledge base ready with 11 documents!


## RAG System Implementation

In [112]:
class ChromaGeminiRAG:
    """
    RAG system using ChromaDB and Google Gemini.
    """

    def __init__(self, collection, model_name):
        self.collection = collection
        self.model = genai.GenerativeModel(model_name)
        print(f"RAG system initialized with {model_name}")

    def retrieve(self, query: str, n_results: int = 3) -> List[Dict]:
        """Retrieve relevant documents from ChromaDB."""
        print(f"Searching for: '{query}'")

        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )

        # Format results into simple list
        docs = []
        if results['ids'] and results['ids'][0]:
            for i in range(len(results['ids'][0])):
                docs.append({
                    'text': results['documents'][0][i],
                    'distance': results['distances'][0][i],
                    'metadata': results['metadatas'][0][i]
                })

        print(f"Found {len(docs)} documents")
        return docs

    def generate(self, query: str, docs: List[Dict]) -> str:
        """Generate answer using retrieved documents."""
        if not docs:
            return "No relevant documents found."

        # Build context from retrieved documents
        context = "\n\n".join([f"Document {i+1}: {doc['text']}"
                               for i, doc in enumerate(docs)])

        # Create prompt
        prompt = f"""Answer the question based on these documents:

        {context}

        Question: {query}

        Answer:"""
        # print(prompt)
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            # Fallback to showing context if generation fails
            return f"Error: {e}\n\nRetrieved context:\n{context}"

    def ask(self, query: str, n_results: int = 3) -> str:
        """
        Complete RAG pipeline: retrieve relevant docs + generate answer.

        Args:
            query: User's question
            n_results: How many documents to retrieve

        Returns:
            Generated answer
        """
        print("=" * 60)

        # Step 1: Retrieve
        docs = self.retrieve(query, n_results)

        # Step 2: Generate
        print("Generating answer...")
        answer = self.generate(query, docs)

        print("\nAnswer:")
        print("-" * 40)
        print(answer)
        print("=" * 60)

        return answer

# Initialize RAG system
rag = ChromaGeminiRAG(collection, gemini_model)
print("RAG system ready!")

RAG system initialized with gemini-2.5-flash
RAG system ready!


## Demo Query

In [113]:
demo_query = "What is artificial intelligence?"

result = rag.ask(demo_query, 3)

Searching for: 'What is artificial intelligence?'
Found 3 documents
Generating answer...

Answer:
----------------------------------------
Artificial Intelligence (AI) is a branch of computer science focused on creating intelligent machines that can perform tasks typically requiring human intelligence. AI systems can learn, reason, and make decisions.


## Interactive Functions

In [84]:
# Interactive functions

def ask_rag(question: str, num_docs: int = 3):
    """
    Ask a question to the RAG system.

    Args:
        question: Your question
        num_docs: Number of documents to retrieve
    """
    return rag.ask(question, num_docs)

def search_only(query: str, num_docs: int = 5):
    """
    Search the knowledge base without generating a response.

    Args:
        query: Search query
        num_docs: Number of documents to return
    """
    result = rag.retrieve(query, num_docs)

    print(f"\nSearch Results for: '{query}'")
    print("=" * 50)

    # If result is a dict with 'documents', extract that list
    if isinstance(result, dict) and 'documents' in result:
        docs = result['documents']
    else:
        docs = result  # assume it's already a list

    for i, doc in enumerate(docs, start=1):
        distance = doc.get('distance', None)
        metadata = doc.get('metadata', {})

        print(f"\nRank {i}", end="")
        if distance is not None:
            print(f" (Distance: {distance:.3f})")
        else:
            print()
        print(f"Category: {metadata.get('category', 'N/A')}")
        print(f"Topic: {metadata.get('topic', 'N/A')}")
        print(f"Content: {doc.get('text', '')}")
        print("-" * 30)

    return result


def show_knowledge_base():
    """
    Display overview of the knowledge base.
    """
    print("Knowledge Base Overview")
    print("=" * 40)
    print(f"Total documents: {collection.count()}")

    # Get all documents to show categories
    all_docs = collection.get()

    # Count by category
    categories = {}
    for metadata in all_docs['metadatas']:
        cat = metadata['category']
        categories[cat] = categories.get(cat, 0) + 1

    print("\nCategories:")
    for category, count in categories.items():
        print(f"   {category}: {count} documents")

    print("\nAvailable Topics:")
    topics = set(m['topic'] for m in all_docs['metadatas'])
    for topic in sorted(topics):
        print(f"   • {topic}")

    return {
        'total_docs': collection.count(),
        'categories': categories,
        'topics': list(topics)
    }

## Usage Guide

### **Basic Usage**

```python
# Ask a question
ask_rag("What is machine learning?")

# Search without generating response
search_only("artificial intelligence")

# Show knowledge base overview
show_knowledge_base()
```

### **Advanced Options**

```python
# Retrieve more documents
ask_rag("Explain RAG", num_docs=5)
```
### **Tips for Better Results**

- **Be specific**: "What is machine learning?" vs "Tell me about ML"
- **Use key terms**: Include technical terms from your domain
- **Try different phrasings**: Same question, different words
- **Adjust document count**: More docs = more context but potentially more noise

### **Understanding the Output**

- **Distance**: Lower = more similar (0 = identical)
- **Rank**: Order of relevance (1 = most relevant)
- **Category/Topic**: Document metadata for organization

## Let's Try It Now!

In [85]:
# Show knowledge base overview
show_knowledge_base()

Knowledge Base Overview
Total documents: 10

Categories:
   AI Fundamentals: 4 documents
   NLP Techniques: 3 documents
   Tools: 2 documents
   Programming: 1 documents

Available Topics:
   • artificial_intelligence
   • chromadb
   • colab
   • deep_learning
   • embeddings
   • llm
   • machine_learning
   • nlp
   • python
   • rag


{'total_docs': 10,
 'categories': {'AI Fundamentals': 4,
  'NLP Techniques': 3,
  'Tools': 2,
  'Programming': 1},
 'topics': ['deep_learning',
  'nlp',
  'python',
  'chromadb',
  'embeddings',
  'artificial_intelligence',
  'colab',
  'machine_learning',
  'llm',
  'rag']}

In [86]:
# Run a query
rag_search_result = ask_rag("What is this RAG system and what can it do?")

Searching for: 'What is this RAG system and what can it do?'
Found 3 documents
Generating answer...

Answer:
----------------------------------------
RAG (Retrieval-Augmented Generation) is a system that combines information retrieval with text generation. It can search a knowledge base for relevant documents and then use this context to generate accurate, informed responses.


In [89]:
rag_search_only_result = search_only("What is this RAG system and what can it do?", 3)

Searching for: 'What is this RAG system and what can it do?'
Found 3 documents

Search Results for: 'What is this RAG system and what can it do?'

Rank 1 (Distance: 1.062)
Category: NLP Techniques
Topic: rag
Content: RAG (Retrieval-Augmented Generation) combines information retrieval with text generation. It searches a knowledge base for relevant documents, then uses this context to generate accurate, informed responses.
------------------------------

Rank 2 (Distance: 1.879)
Category: AI Fundamentals
Topic: deep_learning
Content: Deep Learning uses artificial neural networks with multiple layers to model complex patterns in data. It's particularly effective for image recognition, natural language processing, and speech recognition.
------------------------------

Rank 3 (Distance: 1.896)
Category: Tools
Topic: chromadb
Content: ChromaDB is an open-source vector database designed for AI applications. It provides easy-to-use APIs for storing, querying, and managing embeddings with buil

In [115]:
# Final example: Add a custom document and test it
print("Adding a custom document for demonstration...")

# Add a document about this notebook itself
custom_doc = {
    "id": "notebook_demo",
    "text": "This RAG notebook demonstrates how to build a Retrieval-Augmented Generation system using ChromaDB for vector storage and Google Gemini for response generation. It includes a complete knowledge base about AI concepts and provides interactive functions for learning and experimentation.",
    "category": "Demo",
    "topic": "notebook_example"
}
collection.add(
    documents=[custom_doc["text"]],
    ids=[custom_doc["id"]],
    metadatas=[{"category": custom_doc["category"], "topic": custom_doc["topic"]}]
)
print(f"Added custom document: {custom_doc['id']}")

print(f"\nFinal knowledge base size: {collection.count()} documents")

Adding a custom document for demonstration...
Added custom document: notebook_demo

Final knowledge base size: 11 documents


In [116]:
# Test with the new document
ask_rag("Tell me about this RAG notebook and what it demonstrates")

Searching for: 'Tell me about this RAG notebook and what it demonstrates'
Found 3 documents
Generating answer...

Answer:
----------------------------------------
This RAG notebook demonstrates how to build a Retrieval-Augmented Generation system. It utilizes ChromaDB for vector storage and Google Gemini for response generation. The notebook includes a complete knowledge base about AI concepts and provides interactive functions for learning and experimentation.


'This RAG notebook demonstrates how to build a Retrieval-Augmented Generation system. It utilizes ChromaDB for vector storage and Google Gemini for response generation. The notebook includes a complete knowledge base about AI concepts and provides interactive functions for learning and experimentation.'